In [ ]:
import re
import json

from simple_websocket_server import WebSocketServer, WebSocket

class Controller: 
    def process(self, telemetry):
        return {
            'throttle': 0.05, 
            'steering_angle': -0.05, 
            'mpc_x': [],
            'mpc_y': []
        }
    
def visualize_trajectory():
    return {
        'next_x': [0.0, 10.0, 20.0, 30.0, 40.0],
        'next_y': [0.0, 0.0, 0.0, 0.0, 0.0],
    }
    

class SimulatorClient(WebSocket):
    incoming_pattern = re.compile('42\["telemetry",\s*(.*)\s*\]')
    manual_command = '42["manual",{}]'
    steer_template = '42["steer",%s]'
    
    def connected(self):
        self.controller = Controller()
        
    def handle(self):
        steer_command = self._receive_telemetry(self.data)
        steer_command.update(visualize_trajectory())
        self._send_command(steer_command)
    
    def handle_close(self):
        print("*** Connection closed")
        
    def _receive_telemetry(self, package):
        match_data = self.incoming_pattern.match(package)
        if not match_data: return None
        
        telemetry = json.loads(match_data.group(1))
        return self.controller.process(telemetry)
            
    def _send_command(self, command):
        if not command: 
            response = self.manual_command
        else: 
            response = self.steer_template % json.dumps(command)
        self.send_message(response)
        

class UdacitySimulator: 
    def run(self):
        server = WebSocketServer('', 4567, SimulatorClient)
        try:
            server.serve_forever()
        except KeyboardInterrupt: 
            server.close()


## Simulator runner

In [ ]:
sim = UdacitySimulator()
sim.run()

## Unit tests

In [110]:
import math
import numpy as np

import unittest

def translate(new_center):
    return np.array([
            [1, 0, -new_center[0]],
            [0, 1, -new_center[1]],
            [0, 0, 1]])

def rotate(alpha):
    return np.array([
            [ np.cos(alpha), np.sin(alpha), 0],
            [-np.sin(alpha), np.cos(alpha), 0],
            [ 0,             0,             1]])


class CoordinateSystem:
    def __init__(self, center, alpha):
        self.center = center
        self.alpha = alpha
        self.transform = rotate(alpha) @ translate(center)
    
    def global_to_local(self, pt):
        pt_g = np.array([pt[0], pt[1], 1])
        pt_l = self.transform @ pt_g
        return tuple(pt_l[:-1])
    

    

class CoordinateSystemTest(unittest.TestCase):
    
    def test_translation_only(self):
        local_system = CoordinateSystem((1, 1), 0)
        global_point = (3, 2)
        local_point = local_system.global_to_local(global_point)
        self.assertEqual((2, 1), local_point)
        
    def test_rotation_only(self):
        local_system = CoordinateSystem((0, 0), math.pi / 4)
        global_point = (1, 1)
        local_x, local_y = local_system.global_to_local(global_point)
        self.assertAlmostEqual(np.sqrt(2), local_x)
        self.assertAlmostEqual(0, local_y)
        
    def test_rotation_and_translation(self):
        local_system = CoordinateSystem((1, 1), math.pi / 4)
        global_point = (2, 2)
        local_x, local_y = local_system.global_to_local(global_point)
        self.assertAlmostEqual(np.sqrt(2), local_x)
        self.assertAlmostEqual(0, local_y)
        
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_rotation_and_translation (__main__.CoordinateSystemTest) ... ok
test_rotation_only (__main__.CoordinateSystemTest) ... ok
test_translation_only (__main__.CoordinateSystemTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


In [ ]:
sample_telemetry = {'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 3.102509e-06}

In [27]:
a1 * x.transpose()

matrix([[1],
        [0],
        [0]])

In [72]:
import math

In [73]:
math.pi

3.141592653589793